In [75]:
import requests
import ast
import pandas as pd
import numpy as np
import re

recipe = requests.get('https://openrecipes.s3.amazonaws.com/openrecipes.txt')

print(recipe.status_code)

200


In [76]:
result = recipe.text

result = recipe.text.split('\n')
print(len(result))
# result

1043


In [77]:
raw = []

for i in result[0:1042]:
    raw.append(ast.literal_eval(i))


In [78]:
df = pd.DataFrame(raw)

In [79]:
df.columns

Index(['name', 'ingredients', 'url', 'image', 'cookTime', 'recipeYield',
       'datePublished', 'prepTime', 'description'],
      dtype='object')

In [80]:
df.loc[4]

name                                           Perfect Iced Coffee
ingredients      1 pound Ground Coffee (good, Rich Roast)\n8 qu...
url              http://thepioneerwoman.com/cooking/2011/06/per...
image            http://static.thepioneerwoman.com/cooking/file...
cookTime                                                        PT
recipeYield                                                     24
datePublished                                           2011-06-13
prepTime                                                      PT8H
description      Iced coffee is my life. When I wake up, often ...
Name: 4, dtype: object

In [81]:
df['ingredients'].str.split('\n').str.len().describe()

count    1042.000000
mean       10.709213
std         4.575432
min         1.000000
25%         8.000000
50%        10.000000
75%        13.000000
max        36.000000
Name: ingredients, dtype: float64

In [82]:
all_ingreds= df['ingredients'].str.split('\n')
x = all_ingreds.str.len() # returns len of ingredients for each recipe
x.describe()

count    1042.000000
mean       10.709213
std         4.575432
min         1.000000
25%         8.000000
50%        10.000000
75%        13.000000
max        36.000000
Name: ingredients, dtype: float64

In [83]:
# lets find the reipe with the most ingredients

df['name'][np.argmax(df['ingredients'].str.len())]

'A Nice Berry Pie'

In [84]:
# return a record where the column name is a Nice Berry Pie
list_ = df[df['name'] == 'A Nice Berry Pie']['ingredients']
list_.to_list()

["Flaky Rye Pie Crust\n75 g / v. scant 2/3 cup rye flour\n175g / 1 1/2 cups unbleached all-purpose flour\n1/4 teaspoon fine grain sea salt\n8 oz / 1 cup salted butter\n1/3 cup / 80 ml cold water or beer\nYou can make the crust using the quick and popular food processor technique. BUT I always make it by hand, using the above ingredients, and this technique. If you like a super-puffy crust, do the folding in Pim's instructions 4x. I usually like mine less so, and fold &amp; roll just 2 or 3 times, depending on how the dough is feeling. The pie in the photo was 3x.\nBerry Pie Filling\n2/3 cup / 3.5 oz / 100g natural cane or brown sugar\n1/3 cup / 1.5 oz / 45 g flour (apf or wwp)\n1/4 teaspoon fine grain sea salt\n1/2 teaspoon cinnamon\n2 sprigs thyme (prefer. lemon thyme) ~ 1/2 t.\n2 pounds of berries, (see head notes)\n2 tablespoons lemon juice\n2 tablespoons butter\n1 egg plus 1 tablespoon water, whisked\nLarge grain sugar, for sprinkling\nPreheat your oven to 425F / 220C, with a rack 

In [85]:
df['description'].str.contains('[Bb]reakfast').sum()

11

In [88]:
spice_list = ['salt', 'pepper', 'oregano', 'parsly','basil', 'ginger', 'turmeric', 'cumin', 'cinnamon']
spice_df = pd.DataFrame(dict((spice,df.ingredients.str.contains(spice, re.IGNORECASE)) for spice in spice_list))
spice_df

,salt,pepper,oregano,parsly,basil,ginger,turmeric,cumin,cinnamon
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
1037,True,False,False,False,False,False,False,False,False
1038,True,False,False,False,False,False,False,False,False
1039,False,True,False,False,False,False,False,False,False
1040,True,False,False,False,False,False,False,False,False


In [91]:
# Recipes with a combination of turmeric and basil
selection = spice_df.query('turmeric & basil')
selection

,salt,pepper,oregano,parsly,basil,ginger,turmeric,cumin,cinnamon
641,True,True,False,False,True,False,True,True,False
784,False,True,False,False,True,True,True,True,False


In [92]:
df['name'][selection.index]

641    Sesame Yogurt Pasta Salad
784            Green Curry Broth
Name: name, dtype: object